In [ ]:
!pip install transformers datasets
!pip install torch
!pip install matplotlib
!pip install peft
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, AdamW
from datasets import load_dataset
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True)


def custom_optimizer(model):
    optimizer_grouped_parameters = [
        {
            "params": [param for name, param in model.named_parameters() if 'bias' in name],
            "weight_decay": 0.01
        }
    ]
    return AdamW(optimizer_grouped_parameters, lr=1e-4)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

dataset = load_dataset("glue", "sst2")
small_train_data, small_new_val_data = dataset['train'].shuffle(seed=42).select(range(3000)).train_test_split(test_size=0.1, seed=42).values()
small_val_dataset = dataset['validation'].shuffle(seed=42).select(range(600))

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).to(device)


for name, param in model.named_parameters():
    if 'bias' not in name:
        param.requires_grad = False
    else:
        continue


for dataset in (small_train_data, small_new_val_data, small_val_dataset):
    dataset.map(tokenize_function, batched=True).set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    logging_dir='./logs',
    logging_steps=25,
    save_steps=50,
    save_total_limit=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=8,
    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    resume_from_checkpoint=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_data,
    eval_dataset=small_new_val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(custom_optimizer(model), None)
)

trainer.train()

In [ ]:
train_steps, validation_accuracy = zip(*[(log['step'], log['eval_accuracy']) for log in trainer.state.log_history if 'eval_accuracy' in log])
training_accuracy = [
    trainer.evaluate(eval_dataset=small_train_data.select(range(100)), metric_key_prefix="train")['train_accuracy']
    for _ in train_steps
]


plt.plot(train_steps, training_accuracy, label="Training Accuracy", marker='o')
plt.plot(train_steps, validation_accuracy, label="Validation Accuracy", marker='o')


plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.ylim([0, 1])
plt.title("Training and Validation Accuracy Over Time")
plt.legend()
plt.show()

print(f"Test Accuracy: {trainer.evaluate(eval_dataset=small_val_dataset)['eval_accuracy']}")

